### OCI Data Science - Useful Tips
Everything stored in the <span style="background-color: #d5d8dc ">/home/datascience</span> folder is now stored on your block volume drive. The <span style="background-color: #d5d8dc ">ads-examples</span> folder has moved outside of your working space. Notebook examples are now accessible through a Launcher tab "Notebook Examples" button.
<details>
<summary><font size="2">1. Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">2. OCI Configuration and Key Files Set Up</font></summary><p>Follow the instructions in the getting-started notebook. That notebook is accessible via the "Getting Started" Launcher tab button.</p>
</details>
<details>
<summary><font size="2">3. Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">4. Typical Cell Imports and Settings</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import MLData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">5. Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [10]:
from pathlib import Path
from glob import glob
base_dir = Path("/home/datascience/yolov5/runs/detect/exp5/")
labels_dir = base / "labels"
output_dir = base / "labels2"
labels = glob(str(labels_dir/'*.txt'))
print ("fonud {} labels".format(len(labels)))
print(labels[0])

fonud 183 labels
/home/datascience/yolov5/runs/detect/exp5/labels/66a60636-e7bb-40a7-a5a8-dece76bd214e.txt


In [48]:
import imagesize
import os
images = glob("/home/datascience/test/*.png")

im_sizes = {}
for im in images: 
    imName = os.path.basename(im.replace('.png',''))
    width, height = imagesize.get(im)
    im_sizes[imName] = (width, height)

In [72]:
labels_json = [{'id': 1, 'name': 'Türtaster'}, {'id': 2, 'name': 'Türe offen'}, {'id': 3, 'name': 'Türgriff'}, {'id': 4, 'name': 'Türe'}]
labels_dict = {(l['id']-1): l['name'] for l in labels_json}


def convert_box(b,tName):
    b = [float(x) for x in b]
    b2 = b.copy()
    width, height = im_sizes[tName]
    b2[0] = (b[0]-b[2]/2) * width
    b2[1] = (b[1]-b[3]/2) * height
    b2[2] = (b[0]+b[2]/2) * width
    b2[3] = (b[1]+b[3]/2) * height
    return [str(int(x)) for x in b2]

def convert_annotations(line,tName):
    a,*b, c = line.split(' ')
    a = labels_dict[int(a)]
    b = convert_box(b,tName)
    return f'{a} {c} {" ".join(b)}'

for l in labels:
    detections = Path(l).read_text()
    tName = os.path.basename(l.replace('.txt',''))
    out = ''
    for line in detections.split('\n')[:-1]:
        out+=convert_annotations(line, tName)
        out+='\n'
    with open(output_dir / (tName+'.txt'),'w') as outFile:
        outFile.write(out)
        outFile.close()

In [71]:
!mkdir /home/datascience/yolov5/runs/detect/exp5/labels2/